# Correlación
**Autor:** Martínez Torres Marcos Alfonso, Facultad de Ingeniería, UNAM

Licensed under <a href="https://creativecommons.org/licenses/by-nc-nd/4.0?ref=chooser-v1">Attribution-NonCommercial-NoDerivatives 4.0 International</a>

Se importan las bibliotecas de Python
* *numpy* : Permite crear y gestionar arreglos multidimensionales, junto con una gran colección de funciones matemáticas de alto nivel que operan sobre estos arreglos. El sitio oficial es https://numpy.org/
* *matplotlib* : Permite crear visualizaciones estáticas, animadas e interactivas en 2D y 3D. El sitio oficial es https://matplotlib.org/
* *ipywidgets*: Esta biblioteca proporciona herramientas interactivas para crear widgets en Jupyter Notebooks y JupyterLab. El sitio oficial es https://jupyter.org/

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
import ipywidgets as widgets

ModuleNotFoundError: No module named 'ipywidgets'

## Teoría

### Integral de correlación

La integral de correlación se define como:
$$
\mathscr{R}_{xh}(t)\:=\:\int_{-\infty}^{\infty}{\mathscr{x}(\tau)\mathscr{h}(\tau -t)}d\tau 
$$
Que es una expresión muy similar a la integral de convolución, la única diferencia es que esta operación no se efectúa el plegamiento en ninguna de las dos funciones.

<center><img src="correlacion_acumulada.gif" alt="Integral de correlacion, ejemplo" />

### Evaluación gráfica

In [ ]:
# Función de la primera gráfica
def e(t):
    return np.where((t>=0) & (t<=1),1,0)

# Función de la segunda gráfica
def h(t):
    return np.where((t>=0) & (t<=1),t,0)

Para observar cuál es la diferencia que existe entre la integral entre la integral de convolución y la de correlación, analicemos ambas operaciones gráficamente, para ello desarrolemos la convolución y la correlación de las siguientes funciones:

<center><img src="Paso1.png" alt="Paso 1" />

En ambas operaciones cambiamos la variable t por $\tau$. En la integral de correlación solo solo se cambia la variable h($\tau$)

<center><img src="Paso2.png" alt="Paso 2" />

#### Paso 1: Desplazamiento

A continuación para la correlación desplazamos a la función h($\tau$-t) si el corrimiento es la derecha, y como h($\tau$+t) si es a la izquierda

<center><img src="Paso3.1.png" alt="Paso 3" />
<center><img src="Paso3.2.png" alt="Paso 3" />

#### Paso 2: Multiplicación

Para realizar la correlación, como se establece en la definición, es necesario multiplicar la función $\mathscr{e}(\tau)$ por cada valor desplazado de la función $\mathscr{h}(\tau)$ e integrar el producto. El valor de desplazamiento del desplazamiento t se toma de $-\infty$ a $\infty$.<br>
Para nuestro ejemplo, si tomamos un desplazamiento cada $0.25$, podemos observar graficamente como se va comportando nuestra convolución en estos puntos. Por ejemplo, si se tiene un desplazamiento $t = 0$, el area de las dos funciones es cero.

##### Codigo 1

In [ ]:
# Funciones
def e(t):
    return np.where((t >= 0) & (t <= 1), 1, 0)

def h(t, desp):
    return np.where((t >= -desp) & (t <= 1 - desp), t + desp, 0)

def correlacion_valor(t_val):
    t_int = np.linspace(-1, 2, 1000)
    integrando = e(t_int) * h(t_int, t_val)
    return np.trapz(integrando, t_int)

# Dominio temporal
t_vals = np.linspace(-1, 2, 100)
r_vals = np.array([correlacion_valor(tv) for tv in t_vals])
t_tau = np.linspace(-3, 3, 1000)

# Función para actualización
def actualizar(frame):
    t_shift = t_vals[frame]
    h_t = h(t_tau, t_shift)

    # Gráfica 1: superposición
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    ax1.plot(t_tau, e(t_tau), label='x(τ)', color='blue')
    ax1.plot(t_tau, h_t, label='h(t+τ)', color='red')
    ax1.fill_between(t_tau, e(t_tau) * h_t, color='green', alpha=0.5)
    ax1.set_xlim(-1, 2)
    ax1.set_ylim(0, 1.5)
    ax1.set_xlabel('τ')
    ax1.set_title(f'Superposición en t = {t_shift:.2f}')
    ax1.legend()

    # Gráfica 2: correlación acumulada
    ax2.plot(t_vals[:frame+1], r_vals[:frame+1], label='r(t)', color='purple')
    ax2.set_xlim(-1, 2)
    ax2.set_ylim(0, max(r_vals) + 0.1)
    ax2.set_xlabel('t')
    ax2.set_title('Correlación cruzada $r_{xh}(t)$')
    ax2.legend()

    plt.tight_layout()
    plt.show()

# Widget interactivo
interact(actualizar, frame=IntSlider(min=0, max=len(t_vals)-1, step=1, value=0, description='t'))

C:\Users\Marcos\AppData\Local\Temp\ipykernel_12512\99203504.py:11: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(integrando, t_int)


interactive(children=(IntSlider(value=0, description='t', max=99), Output()), _dom_classes=('widget-interact',…

<function __main__.actualizar(frame)>

### Evaluación analítica

Las funciones $ x(t) $ y $ h(t) $ están definidas como:

$$
x(t) =
\begin{cases}
1 & ;\quad 0 \leq t \leq 1 \\
0 & ;\quad \forall t \text{ fuera de ese intervalo}
\end{cases}
\qquad
h(t) =
\begin{cases}
t & ;\quad 0 \leq t \leq 1 \\
0 & ;\quad \forall t \text{ fuera de ese intervalo}
\end{cases}
$$

La correlación cruzada $ \mathcal{R}_{xh}(t) $ está dada por:

$$
\mathcal{R}_{xh}(t) = \int_{-\infty}^{+\infty} x(\tau)\, h(t - \tau)\, d\tau
$$

Para determinar los límites de integración se considera el soporte de las funciones:

| Función          | Límite inferior | Límite superior |
|------------------|------------------|------------------|
| $ x(\tau) $     | 0                | 1                |
| $ h(t - \tau) $ | $ t $          | $ t + 1 $    |


La función de correlación cruzada $ \mathcal{R}_{xh}(t) $ está definida como:

$$
\mathcal{R}_{xh}(t) =
\begin{cases}
0 \\
\int_0^{t+1} (1)(\tau - t) \, d\tau \\
\int_t^1 (1)(\tau - t) \, d\tau & \\
0 
\end{cases}
$$

Por lo tanto,

$$
\mathcal{R}_{xh}(t) = \int_a^b (1)(\tau - t)\, d\tau = \left[\frac{\tau^2}{2} - t\tau\right]_a^b
$$

Para cada intervalo determinado:

Evaluando:

Para $ -1 < t \leq 0 $:

$$
I_1 = \left[\frac{\tau^2}{2} - t\tau \right]_0^{t+1} =
\frac{(t+1)^2}{2} - t(t+1) = \frac{t^2 + 2t + 1}{2} - t^2 - t = \frac{1 - t^2}{2}
$$

Para $ 0 < t \leq 1 $:

$$
I_2 = \left[\frac{\tau^2}{2} - t\tau \right]_t^1 =
\left( \frac{1}{2} - t \right) - \left( \frac{t^2}{2} - t^2 \right) =
\frac{1 - 2t + t^2}{2} = \frac{(t - 1)^2}{2}
$$

Entonces la función completa de correlación es:

$$
\mathcal{R}_{xh}(t) =
\begin{cases}
0 & ;\quad t \leq -1 \\
\frac{1}{2}(1 - t^2) & ;\quad -1 < t \leq 0 \\
\frac{1}{2}(t - 1)^2 & ;\quad 0 < t \leq 1 \\
0 & ;\quad t > 1
\end{cases}
$$


### Teorema de la correlación

El teorema de la correlación señala la relación existente entre la integral de correlación y la transformada de Fourier. A esta relación se le conoce como el **teorema de la correlación**.

$$
\mathcal{F}\{\mathcal{R}_{xh}(t)\} = \textbf{H}(\mathcal{f}) \textbf{X}^{*}(\mathcal{f})
$$

Esta expresión representa el teorema de la correlación, el cual nos dice que la correlación en el dominio del tiempo de dos funciones es equivalente a la multiplicación de la transformada de Fourier de una función con el conjugado de la transformada de Fourier de la otra, todo en el dominio de la frecuencia.